In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

In [2]:
players_file = open(r'data/players.pkl', 'rb')
players = pickle.load(players_file)

In [3]:
results_file = open(r'data/results.pkl', 'rb')
results = pickle.load(results_file)

In [4]:
tournaments_file = open(r'data/tournaments.pkl', 'rb')
tournaments = pickle.load(tournaments_file)

In [5]:
len(tournaments), len(results), len(players)

(5528, 5528, 204063)

1. Разбиваем на трейн и тест. Также оставляем только турниры следующих типов: 'Обычный', 'Синхрон', 'Строго синхронный'. Они входят по правилам в определение рейтинга, покрывают большую часть турниров и с ними меньше проблем при процессинге. Например, турниры "Общий зачет" могут иметь повторяющиеся вопросы и имея один и тот же норм менять длину маски ответов.

In [6]:
tournament_type_to_keep = ['Обычный', 'Синхрон', 'Строго синхронный']

In [7]:
tournament_type = set()

cnt = 0
for key, elem in tournaments.items():
    try:
        t = elem['type']['name']
        tournament_type.add(t)
        if t in tournament_type_to_keep:
            cnt += 1
    except KeyError:
        continue
cnt / len(tournaments)  # доля от всех турниров

0.9129884225759769

In [8]:
train_ids = [elem['id'] for key, elem in tournaments.items() if elem['dateStart'][:4] == '2019' \
             and elem['type']['name'] in tournament_type_to_keep]
test_ids = [elem['id'] for key, elem in tournaments.items() if elem['dateStart'][:4] == '2020'
             and elem['type']['name'] in tournament_type_to_keep]

Получили маску из турниров в 2019 и 2020 годах. Теперь из results достанем списки команд, участников и маски ответов. Глобальная цель: составить пары игрок-вопрос, а также  правильно ли сделан ответ. Используя one-hot-encoding для игроков и для вопросов, получим матрицу признаков, вектор с фактом правильности ответа используем в качестве таргета. На данном этапе будем считать, что правильный ответ команды соответствует правильности ответа данного каждым игроком. 
К этим данным можно применить логистическую регрессию.


In [9]:
teams_dct = {}
train_dct = {}
test_dct = {}

for i, ids in enumerate(train_ids):
    tournaments_dct = {}
    for team in results[ids]:
        cur_mask = team.get('mask')
        if cur_mask is None or not team['teamMembers']:  # нет членов команды или записи ответов
            continue
        cur_mask = cur_mask.replace('?', '0')  # для простоты, заменим на 0 все неопределенности типа X и ?
        cur_mask = cur_mask.replace('X', '0')
        cur_mask = [int(ans) for ans in cur_mask]
        team_id = team['team']['id']
        members = [member['player']['id'] for member in team['teamMembers']]
        tournaments_dct[team_id] = {}
        tournaments_dct[team_id]['team_id'] = team_id
        tournaments_dct[team_id]['mask'] = cur_mask
        tournaments_dct[team_id]['players'] = members
    train_dct[ids] = tournaments_dct
    
# аналогично для теста
for i, ids in enumerate(test_ids):
    tournaments_dct = {}
    for team in results[ids]:
        cur_mask = team.get('mask')
        if cur_mask is None or not team['teamMembers']:  
            continue
        cur_mask = cur_mask.replace('?', '0') 
        cur_mask = cur_mask.replace('X', '0')
        cur_mask = [int(ans) for ans in cur_mask]
        team_id = team['team']['id']
        members = [member['player']['id'] for member in team['teamMembers']]
        tournaments_dct[team_id] = {}
        tournaments_dct[team_id]['team_id'] = team_id
        tournaments_dct[team_id]['mask'] = cur_mask
        tournaments_dct[team_id]['players'] = members
    test_dct[ids] = tournaments_dct
        

In [11]:
train = []
qnt = 0

for tournament_id, teams in train_dct.items():
    for team_id, team in teams.items():
        mask = np.array([np.int32(answer) for answer in team['mask']])
        players = team['players']
        questions = np.tile(np.arange(qnt, qnt + len(mask)), len(players))
        ans = np.array(np.meshgrid(players, mask)).T.reshape(-1, 2)
        ans = np.hstack([np.repeat(tournament_id, len(questions)).reshape(-1, 1),
np.repeat(team_id, len(questions)).reshape(-1, 1), questions.reshape(-1, 1), ans])
        train.append(ans)    
    qnt += len(mask)

In [12]:
train = np.vstack(train).astype(np.int32)
train = pd.DataFrame(train,columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'answer'])

In [14]:
test = []
qnt = 0

for tournament_id, teams in test_dct.items():
    for team_id, team in teams.items():
        mask = np.array([np.int32(answer) for answer in team['mask']])
        players = team['players']
        questions = np.tile(np.arange(qnt, qnt + len(mask)), len(players))
        ans = np.array(np.meshgrid(players, mask)).T.reshape(-1, 2)
        ans = np.hstack([np.repeat(tournament_id, len(questions)).reshape(-1, 1),
np.repeat(team_id, len(questions)).reshape(-1, 1), questions.reshape(-1, 1), ans])
        test.append(ans)    
    qnt += len(mask)
    
test = np.vstack(test).astype(np.int32)
test = pd.DataFrame(test, columns = ['tournament_id', 'team_id', 'player_id', 'question_id', 'answer'])

In [28]:
max(train['player_id'])

29971

In [16]:
test = test[test['player_id'].isin(set(train['player_id']))] # оставим в тесте только тех, кто в треине был

In [17]:
encoder = OneHotEncoder(handle_unknown='ignore')

X_train = encoder.fit_transform(train[['player_id', 'question_id']])
y_train = train['answer']

In [24]:
model = LogisticRegression(C=1, random_state=42, n_jobs=-1)

model.fit(X_train, y_train)

/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=1, n_jobs=-1, random_state=42)

In [25]:
X_test = encoder.transform(test[['player_id', 'question_id']])
y_test = test['answer']

In [26]:
model.score(X_test, y_test)

0.5484275824671562

In [29]:
model.coef_

array([[ 2.27681378,  1.66225025,  0.13789691, ..., -0.29170918,
        -0.42771123, -0.01227714]])